<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

1.1 Импортируем библиотекки

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', -1)

1.2 Прочитаем файл и изучим данные

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Выводы: 
-  столбцы RowNumber,CustomerId,Surname не нужны для исследования, удалить;
-  в столбце Tenure есть пропуски, нужно будет исправить и проверить;
-  привести названия столбцов в соответствующий вид;
-  фрейм содержит качественные и количественные признаки;
-  масштаб признаков разный поэтому нужно будет масштабировать признаки.

1.3 Предобработка данных

In [5]:
#названия столбцов приведем к нижниму регистру и разделим "_".
data.columns = data.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]
data.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [6]:
data['geography'] = data['geography'].str.lower()
data['gender'] = data['gender'].str.lower()
data.head()

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,france,female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,spain,female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,france,female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,france,female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,spain,female,43,2.0,125510.82,1,1,1,79084.10,0


In [7]:
#разберемся с пропусками tenure
data['tenure'].isnull().sum()

909

In [8]:
data['tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

Значения распределены от 1 до 10. Можно заполнить пропуски средним или медианой, а можно 0, посмторим что лучше.

In [9]:
print('Среднее арифметическое:', data['tenure'].mean())
print('Медиана:', data['tenure'].median())

Среднее арифметическое: 4.997690023099769
Медиана: 5.0


Среднее и медиана близки друг к другу, но их значения дли признака, который показывает, сколько лет клиент пользуется банком слишком высоки. Это может повлиять на предсказания модели, и не известно как получились пропуски. Возможно клиент оставил эту графу пустой. На мой взгляд, лучший вариант заменить нулями.

In [10]:
#заменим пропуски и изменим тип данных на целый
data['tenure'] = data['tenure'].fillna(0).astype('int')
data['tenure'].value_counts()

0     1291
1     952 
2     950 
8     933 
3     928 
5     927 
7     925 
4     885 
9     882 
6     881 
10    446 
Name: tenure, dtype: int64

In [11]:
#удалим не нужные признаки
df = data.drop(['row_number', 'customer_id', 'surname'], axis=1)
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,france,female,42,2,0.00,1,1,1,101348.88,1
1,608,spain,female,41,1,83807.86,1,0,1,112542.58,0
2,502,france,female,42,8,159660.80,3,1,0,113931.57,1
3,699,france,female,39,1,0.00,2,0,0,93826.63,0
4,850,spain,female,43,2,125510.82,1,1,1,79084.10,0


Осталось 11 признаков: 2 качественных (geography, gender) и 9 количественных.

In [12]:
#проверим дубликаты
df.duplicated().sum()

0

1.4 Преобразование признаков

Преобразуем категориальные признаки в численные с помощью get_dummies() с аргументом drop_first, чтобы не создавать фиктивные признаки.

In [13]:
df_ohe = pd.get_dummies(df, drop_first=True)
df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_germany,geography_spain,gender_male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


1.5 Разобъем данные на выборки

Так как спрятанной тестовой выборки у нас нету, разделим исходные данные на три выборки: обучающую, валидационную и тестовую. Так как размеры тестовой и валидационной выборок обычно равны, разделим данные в соотношении 3:1:1.

Сначала методом train_test_split разделим исходные данные на обучающую (60%) и валидационную выборку (40%). После этого разделим валидационную выборку пополам — на валидационную и тестовую выборки. Таким образом каждая из этих 2-х выборок составит 20% из всех данных df_ohe.

In [14]:
target = df_ohe['exited']
features = df_ohe.drop('exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              train_size=0.60, 
                                                                              test_size=0.40, 
                                                                              random_state=123, 
                                                                              stratify=target)

In [15]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, 
                                                                            target_valid, 
                                                                            train_size=0.50, 
                                                                            test_size=0.50, 
                                                                            random_state=123, 
                                                                            stratify=target_valid)

Проверим результат

In [16]:
samples = {'Размер обучающей выборки' : features_train, 
          'Размер валидационной выборки': features_valid, 
          'Размер тестовой выборки': features_test}

for key, value in samples.items():
    print(key + ':', value.shape)

Размер обучающей выборки: (6000, 11)
Размер валидационной выборки: (2000, 11)
Размер тестовой выборки: (2000, 11)


1.6 Масштабирование признаков.

In [17]:
df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_germany,geography_spain,gender_male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [18]:
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_germany,geography_spain,gender_male
6255,-1.081763,1.561078,-1.463231,0.567612,2.552230,1,0,-1.444757,1,0,1
7141,1.083241,-0.091611,-1.140673,0.097622,-0.926107,0,1,-1.559126,1,0,0
3824,0.167278,0.491691,-1.463231,0.751314,-0.926107,0,0,-1.476028,0,0,1
1901,1.509997,2.727681,0.149559,-1.219982,0.813061,0,1,-0.786010,0,1,1
2886,-2.101813,-0.188828,0.472117,-1.219982,0.813061,1,0,-0.312019,0,0,0


Привели признаки к одному масштабу.

Результат:
- изучили данные и сделали предобработку данных (заменили тип данных и заполнили пропуски для признака tenure, проверили данные на дубликаты и переименовали признаки)
- преобразовали категориальные признаки в численные
- разделили исходные данные на три выборки: обучающую, валидационную и тестовую в соотношении 3:1:1.
- масштабировали признаки.

## Исследование задачи

В рамках проекта нам предстоит решить задачу бинарной классификации: спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Но перед тем, как приступить к задаче, необходимо:

- исследовать баланс классов
- обучить модели без учёта дисбаланса.

In [19]:
#посмотрим на баланс классов
df_ohe['exited'].value_counts() / len(df_ohe['exited']) * 100

0    79.63
1    20.37
Name: exited, dtype: float64

классы не сбалансированны

Обучим разные модели без учёта дисбаланса. Так как нам предстоит решить задачу бинарной классификации, рассмотрим следующие модели классификации:

- дерево принятия решений (Decision Tree Classifier)
- случайный лес (Random Forest Classifier)
- логистическая регрессия (Logistic Regression).

In [20]:
models = [LogisticRegression(random_state=123), 
          DecisionTreeClassifier(random_state=123), 
          RandomForestClassifier(random_state=123)]

results_imbalanced = []

for model in models:
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    
    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])
    
    results_imbalanced.append({'Model': model.__class__.__name__, 
                               'accuracy_imb': accuracy, 
                               'f1_imb': f1, 
                               'ROC AUC_imb': roc_auc})

In [21]:
pd.DataFrame(results_imbalanced).style.highlight_max(color = 'lightgreen', axis = 0)

,Model,accuracy_imb,f1_imb,ROC AUC_imb
0,LogisticRegression,0.804,0.302491,0.731555
1,DecisionTreeClassifier,0.7895,0.493381,0.682758
2,RandomForestClassifier,0.856,0.556923,0.84649


Наилучший результат показала модель случайного леса.

Результат:
- проверили и сделали визуализацию баланса классов

- обучили следующие модели без учёта дисбаланса:  
  - дерево принятия решений  
  - случайный лес  
  - логистическая регрессия.  
- для каждой из моделей вывели следующие метрики: accuracy_score, f1_score и roc_auc_score. По всем метрикам наилучший результат по показала модель случайного леса (а1_score = 0.84, f1_score = 0.53 roc_auc_score = 0.83).

## Борьба с дисбалансом

Существует несколько способов борьбы с дисбалансом, в рамках этого проекта будем применять следующие:

- взвешивание классов (объектам редкого класса придается больший вес)
- увеличение выборки (upsampling)
- уменьшение выборки (downsampling).

3.1. Взвешивание классов

Обучим модели, указывая гиперпараметр class_weight = balanced.

In [22]:
models_balanced = [LogisticRegression(random_state=123, class_weight='balanced'), 
                   DecisionTreeClassifier(random_state=123, class_weight='balanced'), 
                   RandomForestClassifier(random_state=123, class_weight='balanced')]

results_balanced = []

for model in models_balanced:
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    
    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])
    
    results_balanced.append({'Model': model.__class__.__name__,
                             'accuracy_balanced': accuracy,
                             'f1_balanced': f1, 
                             'ROC AUC score_balanced': roc_auc})

In [23]:
final_results = pd.DataFrame(results_imbalanced).merge(pd.DataFrame(results_balanced), on='Model')
final_results = final_results.reindex(sorted(final_results.columns), axis=1)
final_results.style.highlight_max(color = 'lightgreen', axis = 0)

,Model,ROC AUC score_balanced,ROC AUC_imb,accuracy_balanced,accuracy_imb,f1_balanced,f1_imb
0,LogisticRegression,0.734507,0.731555,0.6915,0.804,0.460192,0.302491
1,DecisionTreeClassifier,0.679581,0.682758,0.7975,0.7895,0.491844,0.493381
2,RandomForestClassifier,0.840879,0.84649,0.855,0.856,0.544025,0.556923


Наилучший результат снова у модели случайного леса

3.2. Увеличение выборки (upsampling)

In [24]:
#напишем функцию для увеличения выборки
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=123)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [25]:
# проверим размеры выборок
print(features_upsampled.shape)
print(target_upsampled.shape)

(9666, 11)
(9666,)


In [26]:
target_upsampled.value_counts() / len(target_upsampled) * 100

1    50.569005
0    49.430995
Name: exited, dtype: float64

Выборка сбалансированна можно обучить модели

In [27]:
results_upsampling = []

for model in models:
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    
    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])
    
    results_upsampling.append({'Model': model.__class__.__name__, 
                               'accuracy_up': accuracy,
                               'f1_up': f1, 
                               'ROC AUC score_up': roc_auc})
final_results = final_results.merge(pd.DataFrame(results_upsampling), on='Model')
final_results = final_results.reindex(sorted(final_results.columns), axis=1)
final_results.style.highlight_max(color = 'lightgreen', axis = 0)

,Model,ROC AUC score_balanced,ROC AUC score_up,ROC AUC_imb,accuracy_balanced,accuracy_imb,accuracy_up,f1_balanced,f1_imb,f1_up
0,LogisticRegression,0.734507,0.734518,0.731555,0.6915,0.804,0.6865,0.460192,0.302491,0.458081
1,DecisionTreeClassifier,0.679581,0.682001,0.682758,0.7975,0.7895,0.797,0.491844,0.493381,0.495025
2,RandomForestClassifier,0.840879,0.84209,0.84649,0.855,0.856,0.8515,0.544025,0.556923,0.597015


Видим, что модель случайного леса стала прогнозировать лучше. Сейчас f1-score = 0.57, а roc_auc_score = 0.815 (что ниже, чем на несбалансированной выборке).
  
Модель логистической регрессии лучше отработала на данных, сбалансированных при помощи гиперпараметра class_weight = balanced. При этом показатель roc_auc_score почти не изменился.
  
Дерево принятия решений показывает почти одинаковый результат f1_score на данных, сбалансированных при помощи увеличение выборки (0.49) и на несбалансированных данных (0.49).

3.3. Уменьшение выборки downsampling

In [28]:
# используем функцию для уменьшения частоты объектов частого класса
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=123)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=123)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=123)
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [29]:
# проверим размеры выборок
print(features_downsampled.shape)
print(target_downsampled.shape)

(2416, 11)
(2416,)


In [30]:
target_downsampled.value_counts() / len(target_upsampled) * 100

1    12.642251
0    12.352576
Name: exited, dtype: float64

In [31]:
# обучим модели
results_downsampling = []

for model in models:
    model.fit(features_downsampled, target_downsampled)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    
    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])
    
    results_downsampling.append({'Model': model.__class__.__name__, 
                                 'accuracy_down': accuracy, 
                                 'f1_down': f1, 
                                 'ROC AUC score_down': roc_auc})
final_results = final_results.merge(pd.DataFrame(results_downsampling), on='Model')
final_results = final_results.reindex(sorted(final_results.columns), axis=1)
final_results.style.highlight_max(color = 'lightgreen', axis = 0)

,Model,ROC AUC score_balanced,ROC AUC score_down,ROC AUC score_up,ROC AUC_imb,accuracy_balanced,accuracy_down,accuracy_imb,accuracy_up,f1_balanced,f1_down,f1_imb,f1_up
0,LogisticRegression,0.734507,0.733288,0.734518,0.731555,0.6915,0.682,0.804,0.6865,0.460192,0.45641,0.302491,0.458081
1,DecisionTreeClassifier,0.679581,0.692069,0.682001,0.682758,0.7975,0.6955,0.7895,0.797,0.491844,0.479042,0.493381,0.495025
2,RandomForestClassifier,0.840879,0.840597,0.84209,0.84649,0.855,0.758,0.856,0.8515,0.544025,0.555147,0.556923,0.597015


И снова побеждает модель случайного леса

Видим, что модель логистической регрессии лучше работает на данных, сбалансированных при помощи гиперпараметра class_weight = balanced (f1_score = 0.46). Остальные модели лучше работают на данных, преобразованных при помощи увеличение выборки.
  
На следующем этапе проекта обучим каждую из моделей на тех данных, на которых она показала наилучший результат метрики f1_score. Для каждой модели подберем разные гиперпараметры и попробуем улучшить ее работу.

3.4. Настройка гиперпараметров

Логистическая регрессия

In [32]:
results_lr = []

penalty_l1 = LogisticRegression(random_state=123, class_weight='balanced', solver='liblinear', penalty='l1') 
penalty_l2 = LogisticRegression(random_state=123, class_weight='balanced', solver='lbfgs', penalty='l2') 

models_lr = [penalty_l1, penalty_l2]

for model in models_lr:
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)

    accuracy = accuracy_score(target_valid, predictions_valid)
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])
    
    results_lr.append({'Model': model.__class__.__name__ + '_' + model.penalty , 
                       'Hyperparameters': {'random_state': 123, 
                                          'class_weight': model.class_weight, 
                                          'solver': model.solver, 
                                          'penalty': model.penalty},
                       'Accuracy': accuracy, 
                       'F1 score': f1, 
                       'ROC AUC score': roc_auc})
    
pd.DataFrame(results_lr)

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
0,LogisticRegression_l1,"{'random_state': 123, 'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l1'}",0.758,0.555147,0.734554
1,LogisticRegression_l2,"{'random_state': 123, 'class_weight': 'balanced', 'solver': 'lbfgs', 'penalty': 'l2'}",0.758,0.555147,0.734507


Результат почти идентичен, но модель логистической регрессии с гиперпараметром penalty = l1 дает незначительно лучше результат по метрике roc_auc_score. Результат метрики f1_score в обоих случаях = 0.539.
  
Сохраним результат модель логистической регрессии с гиперпараметром penalty = l1 в переменной best_results.

In [33]:
best_results = []
best_results.append(pd.DataFrame(results_lr).loc[0])

Дерево принятия решений
  
Помним, что данная модель лучше работала на данных, преобразованных при помощи увеличение выборки. В цикле настроим гиперпараметр max_depth (глубина дерева) и попробуем улучшить работу модели.

In [34]:
results_dtc = []

for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    
    model.fit(features_upsampled, target_upsampled)
    predictions_valid = model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
 
    accuracy = accuracy_score(target_valid, predictions_valid)        
    f1 = f1_score(target_valid, predictions_valid)
    roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])        

    results_dtc.append({'Model': 'DecisionTreeClassifier', 
                        'Hyperparameters': {'random_state': 123, 'max_depth':depth},
                        'Accuracy': accuracy, 
                        'F1 score': f1, 
                        'ROC AUC score': roc_auc})

pd.DataFrame(results_dtc).style.highlight_max(color = 'lightgreen', axis = 0)

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
0,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 1}",0.71,0.462963,0.673835
1,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 2}",0.72,0.496403,0.735933
2,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 3}",0.72,0.496403,0.785846
3,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 4}",0.7485,0.544796,0.811886
4,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 5}",0.7365,0.550725,0.833776
5,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 6}",0.7605,0.559338,0.826147
6,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 7}",0.7625,0.548908,0.813561
7,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 8}",0.7615,0.540905,0.78074
8,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 9}",0.758,0.528265,0.762914
9,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 10}",0.752,0.510848,0.735242


Наилучший результат метрики f1_score модель показывает при глубине дерева 6. Добавим эту модель в переменную best_results.

In [35]:
best_results.append(pd.DataFrame(results_dtc).loc[5])

Случайный лес
  
В цикле настроим гиперпараметры max-depth (глубина деревьев) и n_estimators (количество деревьев в лесу). Результат сохраним в переменной results_rfc_up.

In [36]:
%%time
results_rfc_up = []

for depth in range(1,15):
    
    for estimator in range(10, 101, 10):
        
        model = RandomForestClassifier(random_state=123, 
                                       n_estimators=estimator, 
                                       max_depth=depth) 
        
        model.fit(features_upsampled, target_upsampled)
        predictions_valid = model.predict(features_valid)
        probabilities_valid = model.predict_proba(features_valid)
        
        accuracy = accuracy_score(target_valid, predictions_valid)        
        f1 = f1_score(target_valid, predictions_valid)
        roc_auc = roc_auc_score(target_valid, probabilities_valid[:,1])       
    
        results_rfc_up.append({'Model': 'RandomForestClassifier', 
                               'Hyperparameters': {'random_state': 123, 
                                                   'n_estimators': estimator, 
                                                   'max_depth':depth}, 
                               'Accuracy': accuracy,
                               'F1 score': f1, 
                               'ROC AUC score': roc_auc})

CPU times: user 1min 15s, sys: 61.2 ms, total: 1min 15s
Wall time: 1min 15s


In [37]:
model = RandomForestClassifier(random_state=123)
model.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

In [38]:
param_grid = {
    'max_depth' : range(1,16),
    'n_estimators' : range(10,101,10),
}

In [39]:
grid_rfc = GridSearchCV(model, param_grid, scoring='f1')

In [40]:
%%time
grid_rfc.fit(features_upsampled, target_upsampled)

CPU times: user 5min 28s, sys: 775 ms, total: 5min 29s
Wall time: 5min 32s


GridSearchCV(estimator=RandomForestClassifier(random_state=123),
             param_grid={'max_depth': range(1, 16),
                         'n_estimators': range(10, 101, 10)},
             scoring='f1')

In [41]:
grid_rfc.best_score_

0.9525643407748406

In [42]:
grid_rfc.best_params_

{'max_depth': 15, 'n_estimators': 100}

Лучший результат дает следующая комбинация гиперпараметров:

In [43]:
df_rfc_up = pd.DataFrame.from_dict(results_rfc_up)
df_rfc_up[df_rfc_up['F1 score']==df_rfc_up['F1 score'].max()]

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
91,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 20, 'max_depth': 10}",0.818,0.6,0.848249


In [44]:
best_results.append(pd.DataFrame(df_rfc_up).loc[91])
pd.DataFrame(best_results).style.highlight_max(color = 'lightgreen', axis = 0)

,Model,Hyperparameters,Accuracy,F1 score,ROC AUC score
0,LogisticRegression_l1,"{'random_state': 123, 'class_weight': 'balanced', 'solver': 'liblinear', 'penalty': 'l1'}",0.758,0.555147,0.734554
5,DecisionTreeClassifier,"{'random_state': 123, 'max_depth': 6}",0.7605,0.559338,0.826147
91,RandomForestClassifier,"{'random_state': 123, 'n_estimators': 20, 'max_depth': 10}",0.818,0.6,0.848249


Видим, что и после настройки гиперпараметров наилучший результат показала модель случайного леса. Протестируем работу этой модели на тестовых данных.

Результат:
- применили несколько способов борьбы с дисбалансом (взвешивание классов, увеличение выборки, уменьшение выборки ) и проверили, как преобразование данных влияет на способность модели предсказывать уход клиентов.
  
- выяснили, что наилучший показатель таких метрик как f1_score и roc_auc_score модель случайного леса и модель дерева принятия решений достигается на данных преобразованных при помощи увеличение выборки. До настроики гиперпараметров показатель метрики f1_score следующий:
  
- случайный лес: 0.576
- дерево принятия решений: 0.495
- Модель логистической регрессии лучше работает на данных, сбалансированных при помощи гиперпараметра class_weight = balanced (f1_score = 0.46).
  
Настройка гиперпараметров позволила улучшить показатель метрики f1_score для всех моделей:
  
- случайный лес: 0.6
- дерево принятия решений: 0.559
- логистическая регрессия: 0.539.
  
Протестируем модель случайного леса с гиперпараметрами n_estimators = 20 и max_depth = 10 на тестовой выборке.

## Тестирование модели

Протестируем работу модели случайного леса со следующей комбинацией гиперпараметров: n_estimators = 20 и max_depth = 10.

In [45]:
model_final = RandomForestClassifier(random_state=123, n_estimators=100, max_depth=15)

model_final.fit(features_upsampled, target_upsampled)  
predictions_final = model_final.predict(features_test)
probabilities_final = model_final.predict_proba(features_test)

accuracy_final = accuracy_score(target_test, predictions_final)
f1_final = f1_score(target_test, predictions_final)
roc_auc_final = roc_auc_score(target_test, probabilities_final[:,1])

print('accuracy', accuracy_final)
print('f1 score', f1_final)
print('roc_auc', roc_auc_final)

accuracy 0.8565
f1 score 0.6389937106918239
roc_auc 0.8612102086678357


объединим выборки

In [46]:
features_combined = pd.concat([features_train, features_valid])
target_combined = pd.concat([target_train, target_valid])

увеличим выборку

In [47]:
features_upsampled_combined, target_upsampled_combined = upsample(features_combined, target_combined, 4)

In [48]:
print(features_upsampled_combined.shape)
print(target_upsampled_combined.shape)

(12890, 11)
(12890,)


используем полученные выборки при обучении модели случайного леса с гиперпараметрами n_estimators = 20 и max_depth = 10. Протестируем работу модели на тестовой выборке.

In [49]:
model_combined = RandomForestClassifier(random_state=123, n_estimators=100, max_depth=15) 
#заменил параметры на предложенные grid'ом выйграли около 0.9% стоит ли оно того?
model_combined.fit(features_upsampled_combined, target_upsampled_combined)  

predictions_combined = model_combined.predict(features_test)
probabilities_combined = model_combined.predict_proba(features_test)

accuracy_combined = accuracy_score(target_test, predictions_final)
f1_combined = f1_score(target_test, predictions_combined)
roc_auc_combined = roc_auc_score(target_test, probabilities_combined[:,1])

print('accuracy', accuracy_combined)
print('f1 score', f1_combined)
print('roc_auc', roc_auc_combined)

accuracy 0.8565
f1 score 0.6246913580246913
roc_auc 0.8665506801100021


Построим график ROC-кривой

In [50]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_final[:,1])

# ROC-кривая случайного леса
trace_rf = go.Scatter(x = fpr,y = tpr,
                      name = "Случайный лес: " + str(roc_auc_final), 
                      line = dict(width = 2))

# ROC-кривая случайной модели
trace_random = go.Scatter(x = [0.0, 1.0], y = [0.0, 1.0], 
                          name = 'Случайная модель',
                          line = dict(width = 2, dash = 'dot'))

data = [trace_rf, trace_random]
layout = go.Layout(dict(title = 'ROC-кривая',
                        height = 550, width = 900,
                        xaxis = dict(title = "Ложноположительные ответы (False Positive Rate)"),
                        yaxis = dict(title = "Истинно положительные ответы (True Positive Rate)")))

fig = go.Figure(data, layout=layout)
fig.show()

Проверим финальную модель на адекватность.

In [51]:
strategies = ['stratified', 'most_frequent', 'uniform'] 
  
dummy_results = [] 
for strategy in strategies: 
    dc = DummyClassifier(strategy = strategy, random_state = 42)
    
    dc.fit(features_train, target_train) 
    result = dc.score(features_test, target_test) 
    dummy_results.append({strategy: result}) 

pd.DataFrame(dummy_results).style.highlight_max(color = 'lightgreen', axis = 1)

,stratified,most_frequent,uniform
0,0.664,nan,nan
1,nan,0.7965,nan
2,nan,nan,0.4925


Видим, что модель случайного леса с гиперпараметрами n_estimators = 20 и max_depth = 10 работает лучше, чем стратегии случайного прогнозирования классификатора DummyClassifier().

Вывод.
  
- Проверка модели на тестовой выборке показывает, что модель работает достаточно хорошо — показатель метрики f1_score удалось достигнуть выше, чем было заявлено в требованиях (минимальное требование к этой метрике было 0.59, наша модель дает результат 0.62).
  
- Метрика roc_auc_score = 0.858, что является хорошим результатом.
  
- Модель работает хорошо, цель проекта достигнута.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*